# About this Jupyter Notebook

@author: Yingding Wang

This notebook demonstrates fetching image from a private container registry from a Python KF pipeline SDK v2

In [1]:
import sys

# Install kfp to build a pipeline
* Build KF pipeline with SDK: https://www.kubeflow.org/docs/components/pipelines/sdk/build-pipeline/
* Current KFP SDK version on pypi.org: https://pypi.org/project/kfp/ 

In [2]:
!{sys.executable} -m pip install --upgrade --user kfp==1.8.12

In [3]:
!{sys.executable} -m pip show kubernetes

Name: kubernetes
Version: 12.0.1
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /opt/conda/lib/python3.8/site-packages
Requires: python-dateutil, setuptools, six, google-auth, certifi, requests-oauthlib, urllib3, requests, pyyaml, websocket-client
Required-by: kfp, kfserving


Name: kubernetes
Version: 12.0.1
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /opt/conda/lib/python3.8/site-packages
Requires: pyyaml, requests-oauthlib, setuptools, certifi, six, urllib3, websocket-client, google-auth, requests, python-dateutil
Required-by: kfp, kfserving

## Restart the Kernal

In [4]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.8.10


In [5]:
# run kubectl command line to see the quota in the name space
!kubectl describe quota

Name:                                                                 kf-resource-quota
Namespace:                                                            kubeflow-kindfor
Resource                                                              Used    Hard
--------                                                              ----    ----
cpu                                                                   2090m   18
csi-s3.storageclass.storage.k8s.io/persistentvolumeclaims             0       10
csi-s3.storageclass.storage.k8s.io/requests.storage                   0       2Ti
kubeflow-nfs-csi.storageclass.storage.k8s.io/persistentvolumeclaims   1       20
kubeflow-nfs-csi.storageclass.storage.k8s.io/requests.storage         5Gi     4Ti
memory                                                                4486Mi  140Gi
microk8s-hostpath.storageclass.storage.k8s.io/persistentvolumeclaims  0       5
microk8s-hostpath.storageclass.storage.k8s.io/requests.storage        0       20Gi
minio

## Create K8s secret
* k8s python client API doc: https://github.com/kubernetes-client/python

In [6]:
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config

K8_NAME_SPACE = 'kubeflow-kindfor'
K8_GIT_SECRET_NAME = 'lrzgit-kindfor-playground'

### Construct an empty k8s V1Object Reference

Use the following line to construct an empty K8s V1Object for KFP Python SDK.
```python
secret = k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)
```

In [7]:
secret = k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)

print(type(secret))
print(secret)
print(secret.namespace)

<class 'kubernetes.client.models.v1_object_reference.V1ObjectReference'>
{'api_version': None,
 'field_path': None,
 'kind': None,
 'name': 'lrzgit-kindfor-playground',
 'namespace': 'kubeflow-kindfor',
 'resource_version': None,
 'uid': None}
kubeflow-kindfor


### create a secret with kubectl

1. Create a Deployment Token on GIT LAB with an optional user name, and readonly_registory to pull image
2. SSH Login to your k8s cluster and manually create a docker-registry secret with
```console
kubectl -n kubeflow-kindfor create secret docker-registry <K8_GIT_SECRET_NAME> --docker-server=<GIT_URI> --docker-email=<an_abitrary_email> --docker-password=<deployment_token_string> --docker-username=<deployment_token_user_name>
```

Reference:
* https://kubernetes.io/docs/tasks/configure-pod-container/pull-image-private-registry/

Notice:\
use the following command to examin the docker-registry secret you created in terminal.
```console
kubectl -n <kubeflow_namespace> get secret <K8_GIT_SECRET_NAME> --output="jsonpath={.data.\.dockerconfigjson}" | base64 --decode
```

## Getting started with KubeFlow Python SDK

* Python SDK V2: https://www.kubeflow.org/docs/components/pipelines/sdk-v2/python-function-components/
* Python SDK Overview: https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/

In [8]:
EXPERIMENT_NAME = 'customcontainer'        # Name of the experiment groups runs in the GUI
EXPERIMENT_DESC = 'Test pulling container with kfp SDK from a private image registory'

IMAGE_TAG = "0.0.1"

use_custom_image = True
#use_custom_image = False
if (use_custom_image):
    BASE_IMAGE = f"gitlab.lrz.de:5005/dzkj/playground/demo:{IMAGE_TAG}"
else:
    BASE_IMAGE = "python:3.8.13" # f"library/python:{python_version()}"
print(f"BASE_IMAGE: {BASE_IMAGE}")

BASE_IMAGE: gitlab.lrz.de:5005/dzkj/playground/demo:0.0.1


### V2 Compatible pipeline
* https://www.kubeflow.org/docs/components/pipelines/sdk-v2/v2-compatibility/

In [9]:
import kfp
import kfp.dsl as dsl
# import kfp.v2.dsl.component
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
    ContainerOp
)

In [10]:
# from kfp.dsl import ContainerOp
def pod_resource_transformer(op: ContainerOp, mem_req="200Mi", cpu_req="500m", mem_lim="1000Mi", cpu_lim='2000m'):
    """
    op.set_memory_limit('1000Mi') = 1GB
    op.set_cpu_limit('1000m') = 1 cpu core
    """
    return op.set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)\
            .set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)

In [11]:
def nothing():
    print("hello world!")

# custom_image_op_nothing = kfp.components.create_component_from_func(
custom_image_op_nothing = component(
    func=nothing,
    output_component_file='playground_component.yaml', # This is optional. It saves the component spec for future use.
    base_image=BASE_IMAGE,
    #packages_to_install=['']
)

In [12]:
@dsl.pipeline(
    name = EXPERIMENT_NAME,
    description = EXPERIMENT_DESC
)
def custom_image_pipeline_nothing():
    first_task = pod_resource_transformer(custom_image_op_nothing())
    

### Setting imagePullSecretes in K8s with Pipeline config
* Setting imagePullSecretes for Pipeline with SDK: https://github.com/kubeflow/pipelines/issues/5843#issuecomment-859799181

In [13]:
# from kubernetes import client as k8s_client
pipeline_config = dsl.PipelineConf()
if (use_custom_image):
    pipeline_config.set_image_pull_secrets([k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=K8_NAME_SPACE)])
    pipeline_config.set_image_pull_policy("IfNotPresent")

arguments_nothing = {}

In [14]:
client = kfp.Client()

client.create_run_from_pipeline_func(
    pipeline_func=custom_image_pipeline_nothing,
    arguments = arguments_nothing,
    pipeline_conf=pipeline_config,
    experiment_name=EXPERIMENT_NAME,
    namespace=K8_NAME_SPACE,
    mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE # can not run with V2_COMPATIBLE
)

/home/jovyan/.local/lib/python3.8/site-packages/kfp/compiler/compiler.py:79: UserWarning: V2_COMPATIBLE execution mode is at Beta quality. Some pipeline features may not work as expected.
  warnings.warn('V2_COMPATIBLE execution mode is at Beta quality.'


RunPipelineResult(run_id=5eb024de-a63e-4e45-a05a-2e995a1bc565)